In [ ]:
import sys
sys.path.append('../')

## FiftyOne setup

In [1]:
import os
# connect to global DB instead of local one (adjust path accordingly)
os.environ["FIFTYONE_CONFIG_PATH"] = os.path.expanduser('~') + "/.fiftyone/config.global_mongodb.json"

import fiftyone as fo
from fiftyone import ViewField as F
fo.list_datasets()

['DEBBUG_VIDEOS_DATASET',
 'SAILING_DATASET',
 'SAILING_DATASET_QA',
 'SEGMENTATION_DATASET',
 'SENTRY_IMAGES_SUBSAMPLE',
 'SENTRY_VIDEOS_DATASET_ALL',
 'SENTRY_VIDEOS_DATASET_DEV',
 'SENTRY_VIDEOS_DATASET_QA',
 'SUN_REFLECTION_DATASET',
 'TRAIN_THERMAL_DATASET_2023_06']

In [2]:
# modify at will
det_gt_field = "ground_truth_det"

models = [
    dict(name="tf1zoo_ssd-mobilenet-v2_agnostic_D2207", conf_thr=0),
    dict(name="yolov5n6_RGB_D2304-v1_9C", conf_thr=0),
]

In [3]:
from detection import PrecisionRecallF1Support
from detection.utils import compute_metrics

area_ranges_tuples = [
    ("all", [0, 1e5 ** 2]),
    # ignore anything smaller than 1.53x1.53 in wFoV (6x6 in nFoV + unwated noise)
    ("valid_n", [1.53 ** 2 + 1e-5, 1e5 ** 2]),
    # ignore anything smaller than 6x6 in wFoV (we loose 6x6 --> 18x18 in nFoV)
    ("valid_w", [6 ** 2 + 1e-5, 1e5 ** 2]),
    ("tiny", [0, 1.53 ** 2]),
    ("small", [1.53 ** 2, 6 ** 2]),
    ("medium", [6 ** 2, 12 ** 2]),
    ("large", [12 ** 2, 1e5 ** 2])
]
area_ranges = [v for k, v in area_ranges_tuples]
area_ranges_labels = [k for k, v in area_ranges_tuples]

metric_params = dict(
    iou_thresholds=[1e-10],
    area_ranges=area_ranges,
    area_ranges_labels=area_ranges_labels,
    class_agnostic=True,
)

In [4]:
dataset = fo.load_dataset("SAILING_DATASET_QA")
results = {}
for model in models:
    print(f"##### {model['name']} #####")
    results[model['name']] = compute_metrics(
        view=(dataset
              .select_group_slices(["rgb"])
              .filter_labels(f"{model['name']}", F("confidence") >= model['conf_thr'], only_matches=False)
        ),
        gt_field=det_gt_field,
        pred_field=model['name'],
        metric_fn=PrecisionRecallF1Support,
        metric_kwargs=metric_params
    )
    print()

##### tf1zoo_ssd-mobilenet-v2_agnostic_D2207 #####
Converting to metric format...
Computing metrics...
                                             METRIC     tp,     fp,     fn,    dup,    pr,    re,    f1,   supp,    fpi,  nImgs
@[ IoU=0.00      | area=      all | maxDets=100 ] = 486563, 1605626, 335862, 173987,  0.23,  0.59,  0.33, 822425,  56762, 397322
@[ IoU=0.00      | area=  valid_n | maxDets=100 ] = 486159, 1605645, 332109, 174006,  0.23,  0.59,  0.33, 818268,  56920, 397322
@[ IoU=0.00      | area=  valid_w | maxDets=100 ] = 284185, 248053,  92283,  49391,  0.53,  0.75,  0.63, 376468,  33835, 397322
@[ IoU=0.00      | area=     tiny | maxDets=100 ] =    907,      0,   3250,      0,  1.00,  0.22,  0.36,   4157,      0, 397322
@[ IoU=0.00      | area=    small | maxDets=100 ] = 223225, 1355403, 218575, 122426,  0.14,  0.51,  0.22, 441800,  82523, 397322
@[ IoU=0.00      | area=   medium | maxDets=100 ] = 117541, 173702,  23434,  33229,  0.40,  0.83,  0.54, 140975,  39760, 39732

In [39]:
import plotly.graph_objects as go
from detection.utils import get_confidence_metric_vals

# Create traces
fig = go.Figure()
metrics = ['precision', 'recall', 'f1']
for model in models:
    plot_data = get_confidence_metric_vals(
        cocoeval=results[model['name']]['eval'],
        T=0, R=0, K=0, A=0, M=0)
    for metric in metrics:
        fig.add_trace(
            go.Scatter(
                x=plot_data['conf'],
                y=plot_data[metric],
                mode='lines',
                name=f"{model['name'].split('_')[0]} {metric}",
                line=dict(dash=None if metric == 'f1' else 'dash'),
            )
        )

fig.update_layout(
    title="Metric vs Confidence",
    hovermode='x unified',
    xaxis_title="Confidence",
    yaxis_title="Metric value")
fig.show()

In [7]:
models = [
    dict(name="tf1zoo_ssd-mobilenet-v2_agnostic_D2207", conf_thr=0.27),
    dict(name="yolov5n6_RGB_D2304-v1_9C", conf_thr=0.2),
]

dataset = fo.load_dataset("SAILING_DATASET_QA")
tuned_results = {}
for model in models:
    print(f"##### {model['name']} #####")
    tuned_results[model['name']] = compute_metrics(
        view=(dataset
              .select_group_slices(["rgb"])
              .filter_labels(f"{model['name']}", F("confidence") >= model['conf_thr'], only_matches=False)
        ),
        gt_field=det_gt_field,
        pred_field=model['name'],
        metric_fn=PrecisionRecallF1Support,
        metric_kwargs=metric_params
    )
    print()

##### tf1zoo_ssd-mobilenet-v2_agnostic_D2207 #####


Converting to metric format...
Computing metrics...
                                             METRIC     tp,     fp,     fn,    dup,    pr,    re,    f1,   supp,    fpi,  nImgs
@[ IoU=0.00      | area=      all | maxDets=100 ] = 300268,  90904, 522157,   6446,  0.77,  0.37,  0.49, 822425,  14628, 397322
@[ IoU=0.00      | area=  valid_n | maxDets=100 ] = 300150,  90904, 518118,   6446,  0.77,  0.37,  0.50, 818268,  14701, 397322
@[ IoU=0.00      | area=  valid_w | maxDets=100 ] = 185180,  26556, 191288,   2369,  0.87,  0.49,  0.63, 376468,   9467, 397322
@[ IoU=0.00      | area=     tiny | maxDets=100 ] =    526,      0,   3631,      0,  1.00,  0.13,  0.22,   4157,      0, 397322
@[ IoU=0.00      | area=    small | maxDets=100 ] = 137120,  64561, 304680,   4290,  0.68,  0.31,  0.43, 441800,  16157, 397322
@[ IoU=0.00      | area=   medium | maxDets=100 ] =  83887,  16805,  57088,   1209,  0.83,  0.60,  0.69, 140975,   8595, 397322
@[ IoU=0.00      | area=    large | maxDets=100 ] = 

In [32]:
import numpy as np
import pandas as pd
import plotly.express as px

metrics = ['precision', 'recall', 'f1']
df = pd.DataFrame(dict(
    model=np.concatenate(
        [[f"{model['name']}@{model['conf_thr']}"] * len(metrics) for model in models]),
    score_type=metrics * len(models),
    score=np.concatenate([
        [tuned_results[model['name']]['metrics']['all'][metric]
            for metric in metrics]
        for model in models])
))

fig = px.bar(df, x='score_type', y='score', color='model', barmode='group',
             title="Eval on whole SAILING_DATASET_QA")
fig.update_yaxes(range=[0, 1])
fig.show()

In [11]:
np.concatenate([[
        tuned_results[model['name']]['metrics']['all']['precision'],
        tuned_results[model['name']]['metrics']['all']['recall'],
        tuned_results[model['name']]['metrics']['all']['f1'],
    ] for model in models])

array([0.76761118, 0.36510077, 0.49483972, 0.76517612, 0.5081363 ,
       0.61071228])

In [27]:
models = [
    dict(name="tf1zoo_ssd-mobilenet-v2_agnostic_D2207", conf_thr=0.27),
    dict(name="yolov5n6_RGB_D2304-v1_9C", conf_thr=0.20),
]

dataset = fo.load_dataset("SAILING_DATASET_QA")
offshore_results = {}
for model in models:
    print(f"##### {model['name']} #####")
    offshore_results[model['name']] = compute_metrics(
        view=(dataset
              .select_group_slices(["rgb"])
              .match(F("camera_name").contains_str(["CAM50_CUNX", "ELP_1080"]))
              .filter_labels(f"{model['name']}", F("confidence") >= model['conf_thr'], only_matches=False)
        ),
        gt_field=det_gt_field,
        pred_field=model['name'],
        metric_fn=PrecisionRecallF1Support,
        metric_kwargs=metric_params
    )
    print()

##### tf1zoo_ssd-mobilenet-v2_agnostic_D2207 #####
Converting to metric format...
Computing metrics...
                                             METRIC     tp,     fp,     fn,    dup,    pr,    re,    f1,   supp,    fpi,  nImgs
@[ IoU=0.00      | area=      all | maxDets=100 ] =  66086,  35875, 242714,   1921,  0.65,  0.21,  0.32, 308800,   6308,  61011
@[ IoU=0.00      | area=  valid_n | maxDets=100 ] =  66069,  35875, 239724,   1921,  0.65,  0.22,  0.32, 305793,   6361,  61011
@[ IoU=0.00      | area=  valid_w | maxDets=100 ] =  40777,  15935,  55082,    777,  0.72,  0.43,  0.53,  95859,   5930,  61011
@[ IoU=0.00      | area=     tiny | maxDets=100 ] =    405,      0,   2602,      0,  1.00,  0.13,  0.24,   3007,      0,  61011
@[ IoU=0.00      | area=    small | maxDets=100 ] =  36401,  20018, 173533,   1222,  0.65,  0.17,  0.27, 209934,   4603,  61011
@[ IoU=0.00      | area=   medium | maxDets=100 ] =  15174,  10195,  17186,    339,  0.60,  0.47,  0.53,  32360,   5230,  61011
@

In [36]:
metrics = ['precision', 'recall', 'f1']
df = pd.DataFrame(dict(
    model=np.concatenate(
        [[f"{model['name']}@{model['conf_thr']}"] * len(metrics) for model in models]),
    score_type=metrics * len(models),
    score=np.concatenate([
        [offshore_results[model['name']]['metrics']['all'][metric]
            for metric in metrics]
        for model in models])
))

fig = px.bar(df, x='score_type', y='score', color='model', barmode='group',
             title="Eval on SAILING_DATASET_QA (eFoV + Offshore data ONLY))")
fig.update_yaxes(range=[0, 1])
fig.show()

In [38]:
import plotly.graph_objects as go
from detection.utils import get_confidence_metric_vals

# Create traces
fig = go.Figure()
metrics = ['precision', 'recall', 'f1']
for model in models:
    plot_data = get_confidence_metric_vals(
        cocoeval=offshore_results[model['name']]['eval'],
        T=0, R=0, K=0, A=0, M=0)
    for metric in metrics:
        fig.add_trace(
            go.Scatter(
                x=plot_data['conf'],
                y=plot_data[metric],
                mode='lines',
                name=f"{model['name'].split('_')[0]} {metric}",
                line=dict(dash=None if metric == 'f1' else 'dash'),
            )
        )

fig.update_layout(
    title="Metric vs Confidence",
    hovermode='x unified',
    xaxis_title="Confidence",
    yaxis_title="Metric value")
fig.show()